In [1]:
import random
import heapq
import csv
from time import time
import pygame
import pygame
import heapq

pygame.font.init()


pygame 2.6.1 (SDL 2.28.4, Python 3.11.9)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:

class Matrices:
    def current_matrix(self, n):
        #-- Génère une matrice initiale aléatoire et solvable.
        values = [i + 1 for i in range(n * n - 1)] + [0] #generation matrice [0+1 ,1+1 , ..... ]+[0]
        while True:
            random.shuffle(values) #Mélange la liste
            if self.is_solvable(values, n):
                break
        return [values[i:i + n] for i in range(0, len(values), n)] #decoupage du matrice --> produit : [0, 3, 6]

# values = [1, 2, 3, 4, 5, 6, 7, 8, 0]
# matrice = [
#     [1, 2, 3],
#     [4, 5, 6],
#     [7, 8, 0]
# ]


    def end_matrix(self, n):
        # -- Génère la matrice finale
        values = [i + 1 for i in range(n * n - 1)] + [0]
        return [values[i:i + n] for i in range(0, len(values), n)]


#onditions de solvabilité :
    #Puzzle avec une grille de dimensions impaires (n%2=1)
    #Le puzzle est résolvable si le nombre total d'inversions est pair.
        #Une inversion est une paire de tuiles (𝑎,𝑏) avec 𝑎>𝑏 où 𝑎 apparaît avant b dans l'ordre linéaire du puzzle.

#Puzzle avec une grille de dimensions paires (n%2=0)
    #(nombre d’inversions + ligne du zero)%2=0


#Inversion : Une paire  (a,b) est une inversion si a apparaît avant b dans values, mais a>b (sauf si b=0, car 0 représente une case vide).
    def is_solvable(self, values, n):
        # Vérifie si une configuration de puzzle est solvable.
        inversions = sum(
            1 for i in range(len(values)) for j in range(i + 1, len(values))
            if values[i] > values[j] > 0
        )
        if n % 2 == 1:  # Dimensions impaires
            return inversions % 2 == 0
        else:  # Dimensions paires
            row_empty = values.index(0) // n
            return (inversions + row_empty) % 2 == 0

    #manhattan_distance = ∣x1−x2∣+∣y1−y2∣
    def manhattan_distance(self, etat, goal, n):
        # Calcule la distance de Manhattan entre deux états.
        distance = 0
        for i in range(n):
            for j in range(n):
                value = etat[i][j]
                if value != 0:
                    goal_x, goal_y = divmod(goal.index(value), n)
                    distance += abs(i - goal_x) + abs(j - goal_y)
        return distance


    def get_neighbors(self, etat, n, swap_type="standard", swap_count=1):
        # Génère les voisins selon le type de swap spécifié."""
        flat_etat = sum(etat, [])
        neighbors = []

        if swap_type == "0-swap":
            # Aucun mouvement autorisé
            return []

        elif swap_type == "dynamic":
            # Générer des permutations aléatoires selon le nombre de swaps spécifié
            for _ in range(swap_count):
                new_flat = flat_etat[:]
                idx1, idx2 = random.sample(range(len(new_flat)), 2)
                new_flat[idx1], new_flat[idx2] = new_flat[idx2], new_flat[idx1]
                neighbors.append([new_flat[i:i + n] for i in range(0, len(new_flat), n)])
            return neighbors

        # Cas standard : mouvement de la case vide
        x, y = divmod(flat_etat.index(0), n)
        # déplcement de Droite, gauche, bas, haut
        moves = [(0, 1), (0, -1), (1, 0), (-1, 0)] 

        for dx, dy in moves:
            nx, ny = x + dx, y + dy
            if 0 <= nx < n and 0 <= ny < n:
                new_flat = flat_etat[:]
                new_flat[x * n + y], new_flat[nx * n + ny] = new_flat[nx * n + ny], new_flat[x * n + y]
                neighbors.append([new_flat[i:i + n] for i in range(0, len(new_flat), n)])
        return neighbors
    
    # -- Exporte les valeurs dans un fichier CSV
    def export_results_to_csv(filename, data):
        with open(filename, mode='w', newline='', encoding='utf-8') as file:
            writer = csv.DictWriter(file, fieldnames=data[0].keys())
            writer.writeheader()
            writer.writerows(data)

    def solve_puzzle(self, initial, goal, n, swap_type="standard", swap_count=1):
        #-- Résoud le puzzle avec l'algorithme A* pour un type de swap donné.
        frontier = []
        heapq.heappush(frontier, (0, initial, 0, None))
        explored = set()
        came_from = {}

        flat_goal = sum(goal, [])  # Aplatie la matrice goal pour la comparaison
        while frontier:
            f, current, g, parent = heapq.heappop(frontier)

            current_tuple = tuple(map(tuple, current))
            if current_tuple in explored:
                continue

            came_from[current_tuple] = parent
            explored.add(current_tuple)

            # Vérifier si la solution est trouvée
            if sum(current, []) == flat_goal:  # Aplatie la matrice actuelle pour la comparaison
                path = []
                while current:
                    path.append(current)
                    current = came_from[tuple(map(tuple, current))]
                return path[::-1]

            for neighbor in self.get_neighbors(current, n, swap_type, swap_count):
                neighbor_tuple = tuple(map(tuple, neighbor))
                if neighbor_tuple not in explored:
                    h = self.manhattan_distance(neighbor, flat_goal, n)  
                    heapq.heappush(frontier, (g + 1 + h, neighbor, g + 1, current))
        return None

    @staticmethod
    def print_matrix(matrix):
        # Affiche une matrice
        for row in matrix:
            print(" ".join(f"{val:3}" for val in row))
        print()


In [3]:
# COLORS (r, g, b)
WHITE = (255, 255, 255)
BLACK = (0, 0, 0)
DARKGREY = (40, 40, 40)
LIGHTGREY = (100, 100, 100)
BGCOLOUR = DARKGREY

# game settings
WIDTH = 1100
HEIGHT = 641
FPS = 60
title = 'Puzzle Game'
TILESIZE = 120

#TODO : Specifier GAME_SIZE selon entrée utilisateur

GAME_SIZE = 3

In [4]:
class Tile(pygame.sprite.Sprite):
    def __init__(self, game, x, y, text):
        self.groups = game.all_sprites
        pygame.sprite.Sprite.__init__(self, self.groups)
        self.game = game
        self.image = pygame.Surface((TILESIZE, TILESIZE))
        self.x, self.y = x, y
        self.text = text
        self.rect = self.image.get_rect()
        if self.text != "empty":
            self.font = pygame.font.SysFont("Consolas", 50)
            font_surface = self.font.render(self.text, True, BLACK)
            self.image.fill(WHITE)
            self.font_size = self.font.size(self.text)
            draw_x = (TILESIZE / 2) - self.font_size[0] / 2
            draw_y = (TILESIZE / 2) - self.font_size[1] / 2
            self.image.blit(font_surface, (draw_x, draw_y))
        else:
            self.image.fill(BGCOLOUR)

    def update(self):
        self.rect.x = self.x * TILESIZE
        self.rect.y = self.y * TILESIZE

    def click(self, mouse_x, mouse_y):
        return self.rect.left <= mouse_x <= self.rect.right and self.rect.top <= mouse_y <= self.rect.bottom

    def right(self):
        return self.rect.x + TILESIZE < GAME_SIZE * TILESIZE

    def right4(self):
        return self.rect.x + TILESIZE < 4 * TILESIZE
   
    def left(self):
        return self.rect.x - TILESIZE >= 0

    def up(self):
        return self.rect.y - TILESIZE >= 0

    def down(self):
        return self.rect.y + TILESIZE < GAME_SIZE * TILESIZE

    def down4(self):
        return self.rect.y + TILESIZE < 4 * TILESIZE

class UIElement:
    def __init__(self, x, y, text):
        self.x, self.y = x, y
        self.text = text

    def draw(self, screen):
        font = pygame.font.SysFont("Consolas", 30)
        text = font.render(self.text, True, WHITE)
        screen.blit(text, (self.x, self.y))


class Button:
    def __init__(self, x, y, width, height, text, colour, text_colour):
        self.colour, self.text_colour = colour, text_colour
        self.width, self.height = width, height
        self.x, self.y = x, y
        self.text = text

    def draw(self, screen):
        pygame.draw.rect(screen, self.colour, (self.x, self.y, self.width, self.height))
        font = pygame.font.SysFont("Consolas", 30)
        text = font.render(self.text, True, self.text_colour)
        self.font_size = font.size(self.text)
        draw_x = self.x + (self.width / 2) - self.font_size[0] / 2
        draw_y = self.y + (self.height / 2) - self.font_size[1] / 2
        screen.blit(text, (draw_x, draw_y))

    def click(self, mouse_x, mouse_y):
        return self.x <= mouse_x <= self.x + self.width and self.y <= mouse_y <= self.y + self.height

In [5]:

class Game:
    def __init__(self, game_size):
        pygame.init()
        self.game_size = game_size  # Dimension dynamique du puzzle
        self.screen = pygame.display.set_mode((WIDTH, HEIGHT))
        pygame.display.set_caption(title)
        self.clock = pygame.time.Clock()
        self.shuffle_time = 0
        self.start_shuffle = False
        self.previous_choice = ""
        self.start_game = False
        self.solution_path = []  # Chemin pour le solveur
        self.current_step = 0
        self.solving = False
    

    def current_matrix(self, n):
        values = [i + 1 for i in range(n * n - 1)] + [0]
        while True:
            random.shuffle(values)
            if self.is_solvable(values, n):
                break
        return [values[i:i + n] for i in range(0, len(values), n)]

    def end_matrix(self, n):
        values = [i + 1 for i in range(n * n - 1)] + [0]
        return [values[i:i + n] for i in range(0, len(values), n)]

    def is_solvable(self, values, n):
        inversions = sum(
            1 for i in range(len(values)) for j in range(i + 1, len(values))
            if values[i] > values[j] > 0
        )
        if n % 2 == 1:  # Dimensions impaires
            return inversions % 2 == 0
        else:
            row_empty = values.index(0) // n
            return (inversions + row_empty) % 2 == 0

    def manhattan_distance(self, etat, goal, n):
        distance = 0
        for i in range(n):
            for j in range(n):
                value = etat[i][j]
                if value != 0:
                    goal_x, goal_y = divmod(goal.index(value), n)
                    distance += abs(i - goal_x) + abs(j - goal_y)
        return distance

    def get_neighbors(self, etat, n, swap_type="standard", swap_count=1):
        flat_etat = sum(etat, [])
        neighbors = []

        x, y = divmod(flat_etat.index(0), n)
        moves = [(0, 1), (0, -1), (1, 0), (-1, 0)]  # Droite, Gauche, Bas, Haut

        for dx, dy in moves:
            nx, ny = x + dx, y + dy
            if 0 <= nx < n and 0 <= ny < n:
                new_flat = flat_etat[:]
                new_flat[x * n + y], new_flat[nx * n + ny] = new_flat[nx * n + ny], new_flat[x * n + y]
                neighbors.append([new_flat[i:i + n] for i in range(0, len(new_flat), n)])
        return neighbors


    def create_game(self):
        grid = [[x + y * self.game_size for x in range(1, self.game_size + 1)] for y in range(self.game_size)]
        grid[-1][-1] = 0  # Case vide en bas à droite
        return grid

    def shuffle(self):
        possible_moves = []
        for row, tiles in enumerate(self.tiles):
            for col, tile in enumerate(tiles):
                if tile.text == "empty":
                    if tile.right():
                        possible_moves.append("right")
                    if tile.left():
                        possible_moves.append("left")
                    if tile.up():
                        possible_moves.append("up")
                    if tile.down():
                        possible_moves.append("down")
                    break
            if possible_moves:
                break

        if self.previous_choice == "right":
            possible_moves.remove("left") if "left" in possible_moves else None
        elif self.previous_choice == "left":
            possible_moves.remove("right") if "right" in possible_moves else None
        elif self.previous_choice == "up":
            possible_moves.remove("down") if "down" in possible_moves else None
        elif self.previous_choice == "down":
            possible_moves.remove("up") if "up" in possible_moves else None

        choice = random.choice(possible_moves)
        self.previous_choice = choice
        if choice == "right":
            self.tiles_grid[row][col], self.tiles_grid[row][col + 1] = self.tiles_grid[row][col + 1], self.tiles_grid[row][col]
        elif choice == "left":
            self.tiles_grid[row][col], self.tiles_grid[row][col - 1] = self.tiles_grid[row][col - 1], self.tiles_grid[row][col]
        elif choice == "up":
            self.tiles_grid[row][col], self.tiles_grid[row - 1][col] = self.tiles_grid[row - 1][col], self.tiles_grid[row][col]
        elif choice == "down":
            self.tiles_grid[row][col], self.tiles_grid[row + 1][col] = self.tiles_grid[row + 1][col], self.tiles_grid[row][col]

    def draw_tiles(self):
        self.tiles = []
        for row, x in enumerate(self.tiles_grid):
            self.tiles.append([])
            for col, tile in enumerate(x):
                if tile != 0:
                    self.tiles[row].append(Tile(self, col, row, str(tile)))
                else:
                    self.tiles[row].append(Tile(self, col, row, "empty"))

    def solve_puzzle(self, initial, goal, n, swap_type="standard", swap_count=1):
        frontier = []
        heapq.heappush(frontier, (0, initial, 0, None))
        explored = set()
        came_from = {}

        flat_goal = sum(goal, [])  # Aplatie la matrice goal
        while frontier:
            f, current, g, parent = heapq.heappop(frontier)

            current_tuple = tuple(map(tuple, current))
            if current_tuple in explored:
                continue

            came_from[current_tuple] = parent
            explored.add(current_tuple)

            if sum(current, []) == flat_goal:
                path = []
                while current:
                    path.append(current)
                    current = came_from[tuple(map(tuple, current))]
                return path[::-1]

            for neighbor in self.get_neighbors(current, n, swap_type, swap_count):
                neighbor_tuple = tuple(map(tuple, neighbor))
                if neighbor_tuple not in explored:
                    h = self.manhattan_distance(neighbor, flat_goal, n)
                    heapq.heappush(frontier, (g + 1 + h, neighbor, g + 1, current))
        return None

    def new(self):
        self.all_sprites = pygame.sprite.Group()
        self.tiles_grid = self.create_game()
        self.tiles_grid_completed = self.create_game()
        self.elapsed_time = 0
        self.start_timer = False
        self.start_game = False
        self.buttons_list = [
            Button(500, 100, 200, 50, "Shuffle", WHITE, BLACK),
            Button(500, 170, 200, 50, "Reset", WHITE, BLACK),
            Button(500, 240, 200, 50, "Solver", WHITE, BLACK)
        ]
        self.draw_tiles()

    def run(self):
        self.new()
        self.playing = True
        while self.playing:
            self.clock.tick(FPS)
            self.events()
            self.update()
            self.draw()

    def update(self):
        if self.start_game:
            if self.tiles_grid == self.tiles_grid_completed:
                self.start_game = False

        if self.start_shuffle:
            self.shuffle()
            self.draw_tiles()
            self.shuffle_time += 1
            if self.shuffle_time > 120:
                self.start_shuffle = False
                self.start_game = True
                self.start_timer = True

        if self.solving and self.current_step < len(self.solution_path):
            self.tiles_grid = self.solution_path[self.current_step]
            self.draw_tiles()
            self.current_step += 1
            pygame.time.delay(10)
            if self.current_step >= len(self.solution_path):
                self.solving = False

        self.all_sprites.update()

    def draw(self):
        self.screen.fill(BGCOLOUR)
        self.all_sprites.draw(self.screen)
        self.draw_grid()
        for button in self.buttons_list:
            button.draw(self.screen)
        pygame.display.flip()

    def draw_grid(self):
        for row in range(-1, self.game_size * TILESIZE, TILESIZE):
            pygame.draw.line(self.screen, LIGHTGREY, (row, 0), (row, self.game_size * TILESIZE))
        for col in range(-1, self.game_size * TILESIZE, TILESIZE):
            pygame.draw.line(self.screen, LIGHTGREY, (0, col), (self.game_size * TILESIZE, col))

    def events(self):
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                pygame.quit()
                quit(0)

            if event.type == pygame.MOUSEBUTTONDOWN:
                mouse_x, mouse_y = pygame.mouse.get_pos()
                for row, tiles in enumerate(self.tiles):
                    for col, tile in enumerate(tiles):
                        if tile.click(mouse_x, mouse_y):
                          if self.game_size == 4 :
                            if tile.right4() and self.tiles_grid[row][col + 1] == 0:
                                self.tiles_grid[row][col], self.tiles_grid[row][col + 1] = self.tiles_grid[row][col + 1], self.tiles_grid[row][col]
                            if tile.left() and self.tiles_grid[row][col - 1] == 0:
                                self.tiles_grid[row][col], self.tiles_grid[row][col - 1] = self.tiles_grid[row][col - 1], self.tiles_grid[row][col]
                            if tile.up() and self.tiles_grid[row - 1][col] == 0:
                                self.tiles_grid[row][col], self.tiles_grid[row - 1][col] = self.tiles_grid[row - 1][col], self.tiles_grid[row][col]
                            if tile.down4() and self.tiles_grid[row + 1][col] == 0:
                                self.tiles_grid[row][col], self.tiles_grid[row + 1][col] = self.tiles_grid[row + 1][col], self.tiles_grid[row][col]
                            self.draw_tiles()
                          else :
                            if tile.right() and self.tiles_grid[row][col + 1] == 0:
                                self.tiles_grid[row][col], self.tiles_grid[row][col + 1] = self.tiles_grid[row][col + 1], self.tiles_grid[row][col]
                            if tile.left() and self.tiles_grid[row][col - 1] == 0:
                                self.tiles_grid[row][col], self.tiles_grid[row][col - 1] = self.tiles_grid[row][col - 1], self.tiles_grid[row][col]
                            if tile.up() and self.tiles_grid[row - 1][col] == 0:
                                self.tiles_grid[row][col], self.tiles_grid[row - 1][col] = self.tiles_grid[row - 1][col], self.tiles_grid[row][col]
                            if tile.down() and self.tiles_grid[row + 1][col] == 0:
                                self.tiles_grid[row][col], self.tiles_grid[row + 1][col] = self.tiles_grid[row + 1][col], self.tiles_grid[row][col]
                            self.draw_tiles()

                for button in self.buttons_list:
                    if button.click(mouse_x, mouse_y):
                        if button.text == "Shuffle":
                            self.shuffle_time = 0
                            self.start_shuffle = True
                        if button.text == "Reset":
                            self.new()
                        if button.text == "Solver":
                            path = self.solve_puzzle(self.tiles_grid, self.tiles_grid_completed, self.game_size)
                            if path:
                                self.solution_path = path
                                self.current_step = 0
                                self.solving = True
                            else:
                                print("Pas de solution trouvée.")

Décommander si besoin d'une fichier csv et commenter le code 25

In [6]:
# # Début du programme
# dimension = int(input("Choisir la dimension (3 ou 4) : "))
# swap_type = input("Choisir le type de swap (standard, 0-swap, dynamic) : ").strip().lower()
# swap_count = 1  # Par défaut, un seul swap dynamique

# if swap_type == "dynamic":
#     swap_count = int(input("Spécifiez le nombre de swaps dynamiques : "))

# if dimension in {3, 4} and swap_type in {"standard", "0-swap", "dynamic"}:
#     matrices = Matrices()

#     print("\nMatrice initiale :")
#     start_matrix = matrices.current_matrix(dimension)
#     matrices.print_matrix(start_matrix)

#     print("\nMatrice finale :")
#     goal_matrix = matrices.end_matrix(dimension)
#     matrices.print_matrix(goal_matrix)

#     print("\nRésolution du puzzle :")
#     start_time = time()
#     solution = matrices.solve_puzzle(
#         start_matrix, goal_matrix, dimension, swap_type=swap_type, swap_count=swap_count
#     )
#     end_time = time()

#     execution_time = end_time - start_time

#     if solution:
#         print(f"Solution trouvée en {len(solution) - 1} étapes.")
#         for step_num, step in enumerate(solution):
#             print(f"\nÉtape {step_num + 1} :")
#             matrices.print_matrix(step)
#     else:
#         print("Aucune solution trouvée.")

#     # Collecte des données pour export
#     result = {
#         "Dimension": dimension,
#         "Swap Type": swap_type,
#         "Swap Count": swap_count if swap_type == "dynamic" else None,
#         "Execution Time (s)": execution_time,
#         "Moves": len(solution) - 1 if solution else None,
#         "Status": "Success" if solution else "Failed",
#     }

#     # Export des résultats
#     Matrices.export_results_to_csv("solver_results.csv", [result])
#     print("Les résultats ont été exportés vers 'solver_results.csv'.")

#     # Affichage du contenu du fichier CSV
#     print("\nContenu du fichier CSV :")
#     with open("solver_results.csv", mode="r", encoding="utf-8") as csv_file:
#         reader = csv.reader(csv_file)
#         for row in reader:
#             print(", ".join(row))
# else:
#     print("Veuillez choisir une dimension valide (3 ou 4) et un type de swap correct (standard, 0-swap, dynamic).")

In [ ]:
game_size = int(input("Entrez la taille du puzzle (3 pour 8-puzzle, 4 pour 15-puzzle, etc.) : "))
game = Game(game_size)
while True:
  game.new()
  game.run()

error: display Surface quit

: 